# Merge all Hospital Archives dataframes

In [1]:
import pandas as pd
import numpy as np
import warnings
from IPython.utils import io
import sys
import csv
import os

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

main_dir = os.path.expanduser("~/GitHub/hospitals-data-archive/")


## Load curated/compiled hospital archives data

In [2]:
ls2 = [#'HVBP Total Performance',
       #'HVBP Clinical Outcomes',
       #'HVBP Efficiency',
       #'HVBP Safety',
       #'HVBP HCAHPS',
       'Outpatient Imaging Efficiency',
       'Payment and Value of Care',
       'Complications and Deaths',
       'Hospital Readmissions Reduction Program',
       'Hospital Acquired Conditions Reduction Program',
       #'HCAHPS',
       'Timely and Effective Care',
       'Unplanned Visits',
]


In [3]:
df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hai_df.pkl.gz', compression='gzip')
print('df.shape:', df.shape)
for i in list(df):
    if i not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
        df.rename(columns={i: 'Healthcare Associated Infections: ' + i}, inplace=True)

ls1 = [#'hvbp_total_performance_scores_df.pkl.gz', 
       #'hvbp_clinical_outcomes_df.pkl.gz', 
       #'hvbp_efficiency_df.pkl.gz',
       #'hvbp_safety_df.pkl.gz', 
       #'hvbp_hcahps_df.pkl.gz',
       'Outpatient_Imaging_Efficiency_df.pkl.gz', 
       'Payment_and_ValueOfCare_df.pkl.gz', 
       'complication_and_deaths_df.pkl.gz', 
       'hrrp_df.pkl.gz',
       'hacrp_df.pkl.gz',
       #'hcahps_df.pkl.gz',
       'Timely_and_Effective_Care_df.pkl.gz',
       'unplanned_visits_df.pkl.gz',
      ]

brk = 0
for i, l in enumerate(ls1):
    print(l)
    tdf = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/' + l, compression='gzip')
    
    for ii in list(tdf):
        if ii not in ['Facility ID', 'Facility Name', 'file_month','file_year']:
            tdf.rename(columns={ii: ls2[i] + ': ' + ii}, inplace=True)
    
    df = df.merge(tdf, how='outer')
    print('df.shape:', df.shape)
    
    for iii in list(df):
        if '_x' in iii:
            print('redundant column name:', iii)
            brk = 1
            break
            
    if brk == 1: break
    del tdf

print('df.shape:', df.shape)
df.head()


df.shape: (163645, 54)
Outpatient_Imaging_Efficiency_df.pkl.gz
df.shape: (177692, 61)
Payment_and_ValueOfCare_df.pkl.gz
df.shape: (182213, 77)
complication_and_deaths_df.pkl.gz
df.shape: (503345, 105)
hrrp_df.pkl.gz
df.shape: (524065, 135)
hacrp_df.pkl.gz
df.shape: (530106, 156)
Timely_and_Effective_Care_df.pkl.gz
df.shape: (530242, 308)
unplanned_visits_df.pkl.gz
df.shape: (530242, 373)
df.shape: (530242, 373)


,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Associated Infections: SSI Colon upper CL (HAI-3 CI UPP

## Load general hospital info file from hospitals archives

In [4]:
gen_info_df = pd.read_pickle(main_dir + '/dataframes/partial_dataframes/hospital_info_df.pkl.gz', compression='gzip')
gen_info_df = gen_info_df.filter(items=['Facility ID', 'Hospital Ownership', 'Hospital Type', 'file_month', 'file_year'])
print(gen_info_df.shape)
gen_info_df.drop_duplicates(inplace=True)
print(gen_info_df.shape)

d = {'Acute Care Hospitals': 'Acute Care', 
     np.nan: 'Not given',  
     'Critical Access Hospitals': 'Critical Access', 
     'Acute Care - Department of Defense': 'Acute Care (DOD)',  
     'ACUTE CARE - VETERANS ADMINISTRATION': 'Acute Care (VA)', 
     'Acute Care - Veterans Administration': 'Acute Care (VA)'}
gen_info_df['Hospital Type'].replace(to_replace=d, inplace=True)
print(gen_info_df['Hospital Type'].unique().tolist(), '\n')

d = {'Veterans Health Administration': 'VA', 
     'Government Federal': 'Government - Federal', 
     'Government-State': 'Government - State', 
     'Voluntary non-profit-Other': 'Voluntary non-profit - Other', 
     'Voluntary non-profit-Private': 'Voluntary non-profit - Private', 
     'Government-Federal': 'Government - Federal'}
gen_info_df['Hospital Ownership'].replace(to_replace=d, inplace=True)
print(gen_info_df['Hospital Ownership'].unique().tolist(), '\n')

print(gen_info_df.shape)
gen_info_df = gen_info_df[gen_info_df['Facility ID'].isin(df['Facility ID'].unique())]
print(gen_info_df.shape)

print(len(gen_info_df['Facility ID'].unique()), 'unique hospital IDs')     
gen_info_df.sort_values(by=['Facility ID', 'file_month', 'file_year'], ascending=False, inplace=True)

print(gen_info_df.shape)
gen_info_df.drop_duplicates(subset=['Facility ID'], keep='first', inplace=True)
gen_info_df.drop(labels=['file_month', 'file_year'], axis=1, inplace=True)
print(gen_info_df.shape)
gen_info_df.head()

(231358, 5)
(231358, 5)
['Acute Care', 'Critical Access', 'Childrens', 'Psychiatric', 'Acute Care (DOD)', 'Acute Care (VA)'] 

['Government - Hospital District or Authority', 'Proprietary', 'Voluntary non-profit - Private', 'Government - State', 'Voluntary non-profit - Other', 'Government - Local', 'Voluntary non-profit - Church', 'Government - Federal', 'Tribal', 'Department of Defense', 'Physician', 'VA', 'Government'] 

(231358, 5)
(220030, 5)
5434 unique hospital IDs
(220030, 5)
(5434, 3)


,Facility ID,Hospital Ownership,Hospital Type
5445,670319,Voluntary non-profit - Private,Acute Care
5444,670314,Proprietary,Acute Care
5443,670310,Physician,Acute Care
5442,670309,Proprietary,Acute Care
5441,670300,Voluntary non-profit - Private,Acute Care


## Load file from HCRIS project that contains Lat's, Lon's, and numbers of beds

In [5]:
hcris_df = pd.read_pickle(main_dir + 'Files_For_App/GenDat4App.pkl')
hcris_df.reset_index(drop=True, inplace=True)
hcris_df.columns = hcris_df.columns.droplevel([0,1,2])
hcris_df.rename(columns={'Type of Control of Hospital (See Table I) (S2_1_C1_21)': 'Hospital Ownership',
                     'Hospital type (modified)': 'Hospital Type',
                     'Hospital Provider Number (PRVDR_NUM)': 'Facility ID',
                     'Hospital State (S2_1_C2_2)': 'State',
                     'Total Facility (S3_1_C2_27)': 'Beds',
                    }, inplace=True)

hcris_df = hcris_df.filter(items=['Facility ID', 'Lat', 'Lon', 'Beds', 'State'], axis=1)
hcris_df['Beds'] = hcris_df.groupby(['Facility ID'])['Beds'].transform('mean')
hcris_df['Beds'] = np.round(hcris_df['Beds'], 0)

print(hcris_df.shape)
hcris_df.drop_duplicates(inplace=True)
print(hcris_df.shape)
hosp_ids = df['Facility ID'].unique()
hcris_df = hcris_df[hcris_df['Facility ID'].isin(hosp_ids)]
print(hcris_df.shape)
hcris_df.head()


(73087, 5)
(6908, 5)
(5240, 5)


,Facility ID,Lat,Lon,Beds,State
0,010001,31.214058,-85.361725,377.0,AL
12,010005,NaN,NaN,193.0,AL
24,010006,34.802756,-87.652191,314.0,AL
36,010007,31.291972,-86.255415,64.0,AL
48,010008,31.692595,-86.266156,49.0,AL


## Merge dataframes

In [6]:
print(df.shape)
df = df.merge(gen_info_df, how='outer', on=['Facility ID'])
print(df.shape)
df = df.merge(hcris_df, how='outer', on=['Facility ID'])
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)
df.head()

(530242, 373)
(530242, 375)
(530242, 379)
(530242, 379)


,Facility ID,Facility Name,file_month,file_year,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2 CI LOWER) (Score),Healthcare Associated Infections: CAUTI lower CL (HAI-2a CI LOWER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2 CI UPPER) (Score),Healthcare Associated Infections: CAUTI upper CL (HAI-2a CI UPPER) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF lower CL (HAI-6 CI LOWER) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CDIFF upper CL (HAI-6 CI UPPER) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1 CI LOWER) (Score),Healthcare Associated Infections: CLABSI lower CL (HAI-1a CI LOWER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1 CI UPPER) (Score),Healthcare Associated Infections: CLABSI upper CL (HAI-1a CI UPPER) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA lower CL (HAI-5 CI LOWER) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: MRSA upper CL (HAI-5 CI UPPER) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Abdominal lower CL (HAI-4 CI LOWER) (Score),Healthcare Associated Infections: SSI Abdominal upper CL (HAI-4 CI UPPER) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Healthcare Associated Infections: SSI Colon lower CL (HAI-3 CI LOWER) (Score),Healthcare Associated Infections: SSI Colon upper CL (HAI-3 CI UPP

## Minor formatting

In [7]:
cols = ['State', 'Lon', 'Lat', 'Beds', 'Hospital Ownership', 'Hospital Type', 'file_year', 'file_month', 'Facility Name', 'Facility ID']
for c in cols:
    col = df.pop(c)
    df.insert(0, col.name, col)

labs = list(df)
for c in cols:
    labs.remove(c)
    
print(df.shape)
df = df.dropna(subset=labs, how='all')
print(df.shape)
df.drop_duplicates(subset=cols, inplace=True)
print(df.shape)

df = df[((df['file_month'] == '10') & (df['file_year'].isin(['2020', '2023', '2018', '2021', '2022']))) | ((df['file_month'] == '07') & (df['file_year'].isin(['2013', '2014', '2015', '2016', '2017', '2018', '2019']))) | ((df['file_month'] == '12') & (df['file_year'].isin(['2016'])))] 
print(df.shape)

ls = []
for i in list(df):
    if 'LOWER' in i or 'UPPER' in i or 'lower' in i or 'upper' in i or 'estimate' in i or 'Estimate' in i:
        ls.append(i)

df['Hospital'] = df['Facility Name'] + ' (' + df['Facility ID'] + ')'
df.drop(labels=ls+['file_month', 'Facility ID', 'Facility Name'], axis=1, inplace=True)
col = df.pop('Hospital')
df.insert(0, 'Hospital', col)
print(df.shape)

df.head()

(530242, 379)
(522996, 379)
(219080, 379)
(57773, 379)
(57773, 323)


,Hospital,file_year,Hospital Type,Hospital Ownership,Beds,Lat,Lon,State,Healthcare Associated Infections: CAUTI (SIR) (HAI-2 SIR) (Score),Healthcare Associated Infections: CAUTI Number of Procedures (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2 NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Observed Cases (HAI-2a NUMERATOR) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2 ELIG CASES) (Score),Healthcare Associated Infections: CAUTI Predicted Cases (HAI-2a ELIG CASES) (Score),Healthcare Associated Infections: CAUTI SIR (ICUs only) (HAI-2a SIR) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2 DOPC) (Score),Healthcare Associated Infections: CAUTI Urinary Catheter Days (HAI-2a DOPC) (Score),Healthcare Associated Infections: CDIFF (SIR) (HAI-6 SIR) (Score),Healthcare Associated Infections: CDIFF Observed Cases (HAI-6 NUMERATOR) (Score),Healthcare Associated Infections: CDIFF Predicted Cases (HAI-6 ELIG CASES) (Score),Healthcare Associated Infections: CDIFF patient days (HAI-6 DOPC) (Score),Healthcare Associated Infections: CLABSI (SIR) (HAI-1 SIR) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Device Days (HAI-1a DOPC) (Score),Healthcare Associated Infections: CLABSI Number of Procedures (HAI-1 DOPC) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1 NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Observed Cases (HAI-1a NUMERATOR) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1 ELIG CASES) (Score),Healthcare Associated Infections: CLABSI Predicted Cases (HAI-1a ELIG CASES) (Score),Healthcare Associated Infections: CLABSI SIR (ICUs only) (HAI-1a SIR) (Score),Healthcare Associated Infections: MRSA (SIR) (HAI-5 SIR) (Score),Healthcare Associated Infections: MRSA Observed Cases (HAI-5 NUMERATOR) (Score),Healthcare Associated Infections: MRSA Predicted Cases (HAI-5 ELIG CASES) (Score),Healthcare Associated Infections: MRSA patient days (HAI-5 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR) (Score),Healthcare Associated Infections: SSI Abdominal Number of Procedures (HAI-4 DOPC) (Score),Healthcare Associated Infections: SSI Abdominal Observed Cases (HAI-4 NUMERATOR) (Score),Healthcare Associated Infections: SSI Abdominal Predicted Cases (HAI-4 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Number of Procedures (HAI-3 DOPC) (Score),Healthcare Associated Infections: SSI Colon Observed Cases (HAI-3 NUMERATOR) (Score),Healthcare Associated Infections: SSI Colon Predicted Cases (HAI-3 ELIG CASES) (Score),Healthcare Associated Infections: SSI Colon Surgery (SIR) (HAI-3 SIR) (Score),Outpatient Imaging Efficiency: Abdomen CT Use of Contrast Material (OP-10) (Score),Outpatient Imaging Efficiency: Breast Cancer Screening Recall Rates (OP-39) (Score),Outpatient Imaging Efficiency: MRI Lumbar Spine for Low Back Pain (OP-8) (Score),Outpatient Imaging Efficiency: Mammography Follow-up Rates (OP-9) (Score),Outpatient Imaging Efficiency: Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery (OP-13) (Score),Outpatient Imaging Efficiency: Outpatients with brain CT scans who got a sinus CT scan at the same time (OP-14) (Score),Outpatient Imaging Efficiency: Thorax CT Use of Contrast Material (OP-11) (Score),Payment and Value of Care: Payment for heart attack patients (PAYM_30_AMI) (Denominator),Payment and Value of Care: Payment for heart attack patients (PAYM_30_AMI) (Payment),Payment and Value of Care: Payment for heart failure patients (PAYM_30_HF) (Denominator),Payment and Value of Care: Payment for heart failure patients (PAYM_30_HF) (Payment),Payment and Value of Care: Payment for hip/knee replacement patients (PAYM_90_HIP_KNEE) (Denominator),Payment and Value of Care: Payment for hip/knee replacement patients (PAY

## Make Healthcare Associated Infections files

In [8]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Healthcare Associated Infections:' in i:
        if '1a' in i or '2a' in i:
            continue
        else:
            ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)

measures = []
for i in list(tdf):
    if 'Healthcare Associated Infections:' in i:
        ni = i.replace('Healthcare Associated Infections: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
tdf.rename(columns={'CAUTI (SIR) (HAI-2 SIR)': 'CAUTI (SIR)',
                     'CAUTI Number of Procedures (HAI-2 DOPC)': 'CAUTI Number of Procedures',
                     'CAUTI Observed Cases (HAI-2 NUMERATOR)': 'CAUTI Observed Cases',
                     'CAUTI Predicted Cases (HAI-2 ELIG CASES)': 'CAUTI Predicted Cases',
                     'CAUTI Urinary Catheter Days (HAI-2 DOPC)': 'CAUTI Urinary Catheter Days',
                     'CDIFF (SIR) (HAI-6 SIR)': 'CDIFF (SIR)',
                     'CDIFF Observed Cases (HAI-6 NUMERATOR)': 'CDIFF Observed Cases',
                     'CDIFF Predicted Cases (HAI-6 ELIG CASES)': 'CDIFF Predicted Cases',
                     'CDIFF patient days (HAI-6 DOPC)': 'CDIFF patient days',
                     'CLABSI (SIR) (HAI-1 SIR)': 'CLABSI (SIR)',
                     'CLABSI Number of Device Days (HAI-1 DOPC)': 'CLABSI Number of Device Days',
                     'CLABSI Number of Procedures (HAI-1 DOPC)': 'CLABSI Number of Procedures',
                     'CLABSI Observed Cases (HAI-1 NUMERATOR)': 'CLABSI Observed Cases',
                     'CLABSI Predicted Cases (HAI-1 ELIG CASES)': 'CLABSI Predicted Cases',
                     'MRSA (SIR) (HAI-5 SIR)': 'MRSA (SIR)',
                     'MRSA Observed Cases (HAI-5 NUMERATOR)': 'MRSA Observed Cases',
                     'MRSA Predicted Cases (HAI-5 ELIG CASES)': 'MRSA Predicted Cases',
                     'MRSA patient days (HAI-5 DOPC)': 'MRSA patient days',
                     'SSI Abdominal Hysterectomy (SIR) (HAI-4 SIR)': 'SSI Abdominal Hysterectomy',
                     'SSI Abdominal Number of Procedures (HAI-4 DOPC)': 'SSI Abdominal Number of Procedures',
                     'SSI Abdominal Observed Cases (HAI-4 NUMERATOR)': 'SSI Abdominal Observed Cases',
                     'SSI Abdominal Predicted Cases (HAI-4 ELIG CASES)': 'SSI Abdominal Predicted Cases',
                     'SSI Colon Number of Procedures (HAI-3 DOPC)': 'SSI Colon Number of Procedures',
                     'SSI Colon Observed Cases (HAI-3 NUMERATOR)': 'SSI Colon Observed Cases',
                     'SSI Colon Predicted Cases (HAI-3 ELIG CASES)': 'SSI Colon Predicted Cases',
                     'SSI Colon Surgery (SIR) (HAI-3 SIR)': 'SSI Colon Surgery (SIR)',
                    }, inplace=True)
print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())

for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/HAIs/HAIs_' + yr + '.csv')
    

(44291, 34)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'CAUTI (SIR)', 'CAUTI Number of Procedures', 'CAUTI Observed Cases', 'CAUTI Predicted Cases', 'CAUTI Urinary Catheter Days', 'CDIFF (SIR)', 'CDIFF Observed Cases', 'CDIFF Predicted Cases', 'CDIFF patient days', 'CLABSI (SIR)', 'CLABSI Number of Device Days', 'CLABSI Number of Procedures', 'CLABSI Observed Cases', 'CLABSI Predicted Cases', 'MRSA (SIR)', 'MRSA Observed Cases', 'MRSA Predicted Cases', 'MRSA patient days', 'SSI Abdominal Hysterectomy', 'SSI Abdominal Number of Procedures', 'SSI Abdominal Observed Cases', 'SSI Abdominal Predicted Cases', 'SSI Colon Number of Procedures', 'SSI Colon Observed Cases', 'SSI Colon Predicted Cases', 'SSI Colon Surgery (SIR)']


## Make Outpatient Imaging Efficiency Files

In [9]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Outpatient Imaging Efficiency:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)

measures = []
for i in list(tdf):
    if 'Outpatient Imaging Efficiency:' in i:
        ni = i.replace('Outpatient Imaging Efficiency: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)            

print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Outpatient_Imaging_Efficiency/Outpatient_Imaging_Efficiency_' + yr + '.csv')
    
    

(42901, 15)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'Abdomen CT Use of Contrast Material (OP-10)', 'Breast Cancer Screening Recall Rates (OP-39)', 'MRI Lumbar Spine for Low Back Pain (OP-8)', 'Mammography Follow-up Rates (OP-9)', 'Outpatients who got cardiac imaging stress tests before low-risk outpatient surgery (OP-13)', 'Outpatients with brain CT scans who got a sinus CT scan at the same time (OP-14)', 'Thorax CT Use of Contrast Material (OP-11)']


## Make Payment and Value of Care files

In [10]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Payment and Value of Care:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Payment and Value of Care:' in i:
        ni = i.replace('Payment and Value of Care: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
tdf.rename(columns={'Payment for heart attack patients (PAYM_30_AMI) (Denominator)': 'Payment for heart attack patients (Denominator)', 
                     'Payment for heart attack patients (PAYM_30_AMI) (Payment)': 'Payment for heart attack patients (Payment)', 
                     'Payment for heart failure patients (PAYM_30_HF) (Denominator)': 'Payment for heart failure patients (Denominator)',  
                     'Payment for heart failure patients (PAYM_30_HF) (Payment)': 'Payment for heart failure patients (Payment)',  
                     'Payment for hip/knee replacement patients (PAYM_90_HIP_KNEE) (Denominator)': 'Payment for hip/knee replacement patients (Denominator)',  
                     'Payment for hip/knee replacement patients (PAYM_90_HIP_KNEE) (Payment)': 'Payment for hip/knee replacement patients (Payment)',  
                     'Payment for pneumonia patients (PAYM_30_PN) (Denominator)': 'Payment for pneumonia patients (Denominator)',  
                     'Payment for pneumonia patients (PAYM_30_PN) (Payment)': 'Payment for pneumonia patients (Payment)', 
                    }, inplace=True)
    
print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Payment_and_Value_of_Care/Payment_and_Value_of_Care_' + yr + '.csv')
    

(40081, 16)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'Payment for heart attack patients (Denominator)', 'Payment for heart attack patients (Payment)', 'Payment for heart failure patients (Denominator)', 'Payment for heart failure patients (Payment)', 'Payment for hip/knee replacement patients (Denominator)', 'Payment for hip/knee replacement patients (Payment)', 'Payment for pneumonia patients (Denominator)', 'Payment for pneumonia patients (Payment)']


## Make Complications and Deaths Files

In [12]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Complications and Deaths:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Complications and Deaths:' in i:
        ni = i.replace('Complications and Deaths: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
tdf.rename(columns={'Death rate among surgical inpatients with serious treatable complications (PSI 4)': 'Death rate for surgical inpatients w/ serious treatable complications (PSI 4)', 
                    'Hospital-wide readmission rate after discharge from hospital (READM 30 HOSP WIDE)': 'Hospital-wide readmission rate after discharge (READM 30 HOSP WIDE)', 
                    }, inplace=True)
    
print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Complications_and_Deaths/Complications_and_Deaths_' + yr + '.csv')
    

(42592, 36)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'AMI 30-Day Mortality Rate (MORT 30 AMI)', 'Abdominopelvic accidental puncture or laceration rate (PSI 15)', 'Acute Myocardial Infarction 30-Day Readmission Rate (READM 30 AMI)', 'Death rate for surgical inpatients w/ serious treatable complications (PSI 4)', 'Death rate for CABG surgery patients (MORT 30 CABG)', 'Death rate for COPD patients (MORT 30 COPD)', 'Death rate for stroke patients (MORT 30 STK)', 'Heart failure 30-Day Mortality Rate (MORT 30 HF)', 'Heart failure 30-Day Readmission Rate (READM 30 HF)', 'Hospital-wide readmission rate after discharge (READM 30 HOSP WIDE)', 'Iatrogenic pneumothorax rate (PSI 6)', 'In-hospital fall with hip fracture rate (PSI 8)', 'Infections from a large venous catheter (PSI 7)', 'Patient safety and adverse events composite (PSI 90)', 'Perioperative hemorrhage or hematoma rate (PSI 9)', 'Perioperative pulmonary embolism or deep vein thromb

## Make Hospital Readmissions Reduction Program Files

In [14]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Hospital Readmissions Reduction Program:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Hospital Readmissions Reduction Program:' in i:
        ni = i.replace('Hospital Readmissions Reduction Program: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
    
print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Hospital_Readmissions_Reduction_Program/Hospital_Readmissions_Reduction_Program_' + yr + '.csv')
    

(37763, 38)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'READM-30-AMI (Excess Readmission Ratio)', 'READM-30-AMI (Expected Readmission Rate)', 'READM-30-AMI (Number of Discharges)', 'READM-30-AMI (Number of Readmissions)', 'READM-30-AMI (Predicted Readmission Rate)', 'READM-30-CABG (Excess Readmission Ratio)', 'READM-30-CABG (Expected Readmission Rate)', 'READM-30-CABG (Number of Discharges)', 'READM-30-CABG (Number of Readmissions)', 'READM-30-CABG (Predicted Readmission Rate)', 'READM-30-COPD (Excess Readmission Ratio)', 'READM-30-COPD (Expected Readmission Rate)', 'READM-30-COPD (Number of Discharges)', 'READM-30-COPD (Number of Readmissions)', 'READM-30-COPD (Predicted Readmission Rate)', 'READM-30-HF (Excess Readmission Ratio)', 'READM-30-HF (Expected Readmission Rate)', 'READM-30-HF (Number of Discharges)', 'READM-30-HF (Number of Readmissions)', 'READM-30-HF (Predicted Readmission Rate)', 'READM-30-HIP-KNEE (Excess Readmission 

## Make Hospital Acquired Conditions Reduction Program Files

In [16]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Hospital Acquired Conditions Reduction Program:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Hospital Acquired Conditions Reduction Program:' in i:
        ni = i.replace('Hospital Acquired Conditions Reduction Program: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
    
print(tdf.shape)
print(list(tdf))

years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Hospital_Acquired_Conditions_Reduction_Program/Hospital_Acquired_Conditions_Reduction_Program_' + yr + '.csv')
    

(32579, 29)
['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State', 'CAUTI Score', 'CAUTI SIR', 'CAUTI W Z Score', 'CDI Score', 'CDI SIR', 'CDI W Z Score', 'CLABSI Score', 'CLABSI SIR', 'CLABSI W Z Score', 'MRSA Score', 'MRSA SIR', 'MRSA W Z Score', 'SSI Score', 'SSI SIR', 'SSI W Z Score', 'PSI-90 Score', 'PSI-90 W Z Score', 'Domain 1 Score', 'Domain 2 Score', 'Total HAC Score', 'Payment Reduction']


## Make Timely and Effective Care Files

In [24]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Timely and Effective Care:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Timely and Effective Care:' in i:
        ni = i.replace('Timely and Effective Care: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)

print(tdf.shape)
#for i in list(tdf):
#    print("'" + i + "': " + "'" + i + "',")
        
years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Timely_and_Effective_Care/Timely_and_Effective_Care_' + yr + '.csv')
    

(49197, 160)


## Make Unplanned Visits Files

In [25]:
ls1 = ['Hospital', 'file_year', 'Hospital Type', 'Hospital Ownership', 'Beds', 'Lat', 'Lon', 'State']
ls2 = []
for i in list(df):
    if 'Unplanned Visits:' in i:
        ls2.append(i)

tdf = df.filter(items=ls1 + ls2, axis=1)
measures = []
for i in list(tdf):
    if 'Unplanned Visits:' in i:
        ni = i.replace('Unplanned Visits: ', '')
        tdf.rename(columns={i: ni}, inplace=True)
        
        if ' (Score)' in ni:
            ni2 = ni.replace(' (Score)', '')
            tdf.rename(columns={ni: ni2}, inplace=True)
            measures.append(ni2)
            
        else:
            measures.append(ni)

tdf.dropna(how='all', subset=measures, axis=0, inplace=True)
print(tdf.shape)
        
years = sorted(tdf['file_year'].unique())
for yr in years:
    tdf2 = tdf[tdf['file_year'] == yr]
    tdf2.dropna(how='all', axis=1, inplace=True)
    tdf2.reset_index(drop=True, inplace=True)
    tdf2.to_csv(main_dir + 'dataframes/filtered_files/Unplanned_Visits/Unplanned_Visits_' + yr + '.csv')
    

(31210, 43)
